# Economic Forecasting with Satellite Imagery

This notebook combines satellite-derived activity metrics with traditional economic indicators to build forecasting models.

## Objectives:
1. Load and merge satellite + economic data
2. Feature engineering and correlation analysis
3. Build forecasting models (LSTM, XGBoost, Prophet)
4. Evaluate model performance
5. Make predictions

## Data Sources:
- **Satellite:** Port & retail activity from Sentinel-2 (2017-2024)
- **Economic:** FRED, World Bank, ECB, OECD indicators

## 1. Setup & Imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ Imports successful")

✅ Imports successful


## 2. Load Data

In [5]:
# Paths
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data" / "features"
RESULTS_DIR = PROJECT_ROOT / "results" / "inference_cleaned"

print("📁 Loading data...")

# 1. Satellite inference results (yearly)
satellite_yearly = pd.read_csv(RESULTS_DIR / "summary_by_year_cleaned.csv")
print(f"   ✅ Satellite data: {len(satellite_yearly)} years")

# 2. Economic indicators (monthly)
fred_monthly = pd.read_csv(DATA_DIR / "economic" / "fred_indicators_monthly.csv")
combined_monthly = pd.read_csv(DATA_DIR / "economic" / "combined_economic_indicators_monthly.csv")
oecd_monthly = pd.read_csv(DATA_DIR / "economic" / "oecd_cli_monthly.csv")
print(f"   ✅ Economic data: {len(combined_monthly)} months")

# Display shapes
print(f"\n📊 Data Shapes:")
print(f"   Satellite: {satellite_yearly.shape}")
print(f"   FRED: {fred_monthly.shape}")
print(f"   Combined Economic: {combined_monthly.shape}")
print(f"   OECD: {oecd_monthly.shape}")

📁 Loading data...
   ✅ Satellite data: 10 years


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\MS\\UMD\\Courses\\Fall-2025\\DATA-650\\Real-Time-Economic-Forecasting\\data\\features\\economic\\fred_indicators_monthly.csv'

In [ ]:
# Preview satellite data
print("\n🛰️ Satellite Activity Data:")
display(satellite_yearly)

In [ ]:
# Preview economic data
print("\n💰 Economic Indicators:")
display(combined_monthly.head(10))

In [ ]:
# Check available economic indicators
print("\n📋 AVAILABLE ECONOMIC INDICATORS")
print("="*60)
print(f"Total columns: {len(combined_monthly.columns)}")
print(f"\nColumns in dataset:")
for i, col in enumerate(combined_monthly.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\n✅ Data Coverage:")
print(f"   Years: {combined_monthly['year'].min()} - {combined_monthly['year'].max()}")
print(f"   Months: {len(combined_monthly)} observations")
print(f"   Missing values: {combined_monthly.isnull().sum().sum()}")

## 3. Data Preparation & Merging

In [ ]:
# Load raw inference results (better approach)
inference_raw = pd.read_csv(RESULTS_DIR / "inference_results_cleaned.csv")

# Aggregate to yearly
satellite_clean = inference_raw.groupby('year').agg({
    'total_detections': 'sum',
    'valid_tiles': 'sum'
}).reset_index()

# Rename columns
satellite_clean.columns = ['year', 'satellite_detections', 'valid_tiles']

# Add average per image
satellite_clean['avg_detections_per_image'] = (
    satellite_clean['satellite_detections'] / 
    inference_raw.groupby('year').size().values
)

print("🛰️ Cleaned Satellite Data:")
display(satellite_clean)

In [ ]:
# Aggregate economic data to yearly - WITH ALL AVAILABLE INDICATORS
econ_yearly = combined_monthly.groupby('year').agg({
    # Core Economic Indicators
    'GDP': 'mean',
    'GDP_Growth': 'mean',
    'Unemployment_Rate': 'mean',
    'Industrial_Production': 'mean',
    
    # Consumer & Retail
    'Retail_Sales': 'mean',
    'Consumer_Sentiment': 'mean',
    'Personal_Consumption': 'mean',
    
    # Prices & Housing
    'CPI': 'mean',                      # NEW: Inflation
    'Housing_Starts': 'mean',           # NEW: Housing activity
    
    # Manufacturing & Trade
    'Manufacturing_PMI': 'mean',
    'Employment': 'mean',
    'Exports': 'mean',
    'Imports': 'mean',
    'Trade_Balance': 'mean',
    
    # Financial Markets
    'S&P_500': 'mean',
    'Interest_Rate': 'mean',
    'VIX': 'mean',
    
    # European Central Bank
    'ECB_Deposit_Rate': 'mean',
    'ECB_Refi_Rate': 'mean'
}).reset_index()

# Rename for clarity
econ_yearly = econ_yearly.rename(columns={
    'Unemployment_Rate': 'Unemployment',
    'S&P_500': 'SP500'
})

print("💰 Aggregated Economic Data (Yearly):")
print(f"   Shape: {econ_yearly.shape}")
print(f"   Years: {econ_yearly['year'].min()}-{econ_yearly['year'].max()}")
print(f"   Total indicators: {len(econ_yearly.columns) - 1}")  # -1 for year column
display(econ_yearly)

In [ ]:
# Aggregate economic data to yearly - USING ONLY AVAILABLE COLUMNS
econ_yearly = combined_monthly.groupby('year').agg({
    # Core Economic Indicators
    'GDP': 'mean',
    'GDP_Growth': 'mean',
    'Unemployment_Rate': 'mean',
    'Industrial_Production': 'mean',
    
    # Consumer & Retail
    'Retail_Sales': 'mean',
    'Consumer_Sentiment': 'mean',
    'Personal_Consumption': 'mean',
    
    # Manufacturing & Trade
    'Manufacturing_PMI': 'mean',
    'Employment': 'mean',
    'Exports': 'mean',
    'Imports': 'mean',
    'Trade_Balance': 'mean',
    
    # Financial Markets
    'S&P_500': 'mean',
    'Interest_Rate': 'mean',
    'VIX': 'mean',
    
    # European Central Bank
    'ECB_Deposit_Rate': 'mean',
    'ECB_Refi_Rate': 'mean'
}).reset_index()

# Rename for clarity
econ_yearly = econ_yearly.rename(columns={
    'Unemployment_Rate': 'Unemployment',
    'S&P_500': 'SP500'
})

print("💰 Aggregated Economic Data (Yearly):")
print(f"   Shape: {econ_yearly.shape}")
print(f"   Years: {econ_yearly['year'].min()}-{econ_yearly['year'].max()}")
display(econ_yearly)

## 4. Exploratory Data Analysis

In [ ]:
# Correlation matrix
# ⭐ MERGE SATELLITE + ECONOMIC DATA ⭐
df_combined = satellite_clean.merge(econ_yearly, on='year', how='inner')

print("✅ DATA MERGED SUCCESSFULLY!")
print(f"   Shape: {df_combined.shape}")
print(f"   Years: {df_combined['year'].min()}-{df_combined['year'].max()}")
print(f"   Total features: {len(df_combined.columns)}")

print("\n📊 Combined Dataset:")
display(df_combined)
plt.figure(figsize=(12, 10))
corr = df_combined.drop('year', axis=1).corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Matrix: Satellite Activity vs Economic Indicators', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Time series plots
fig, axes = plt.subplots(3, 2, figsize=(16, 12))

# Satellite detections
axes[0, 0].plot(df_combined['year'], df_combined['satellite_detections'], marker='o', linewidth=2)
axes[0, 0].set_title('Satellite Activity', fontweight='bold')
axes[0, 0].set_ylabel('Total Detections')
axes[0, 0].grid(True, alpha=0.3)

# GDP
axes[0, 1].plot(df_combined['year'], df_combined['GDP'], marker='o', linewidth=2, color='green')
axes[0, 1].set_title('GDP', fontweight='bold')
axes[0, 1].set_ylabel('GDP')
axes[0, 1].grid(True, alpha=0.3)

# Unemployment
axes[1, 0].plot(df_combined['year'], df_combined['Unemployment'], marker='o', linewidth=2, color='red')
axes[1, 0].set_title('Unemployment', fontweight='bold')
axes[1, 0].set_ylabel('Rate (%)')
axes[1, 0].grid(True, alpha=0.3)

# Retail Sales
axes[1, 1].plot(df_combined['year'], df_combined['Retail_Sales'], marker='o', linewidth=2, color='purple')
axes[1, 1].set_title('Retail Sales', fontweight='bold')
axes[1, 1].set_ylabel('Index')
axes[1, 1].grid(True, alpha=0.3)

# Industrial Production
axes[2, 0].plot(df_combined['year'], df_combined['Industrial_Production'], marker='o', linewidth=2, color='orange')
axes[2, 0].set_title('Industrial Production', fontweight='bold')
axes[2, 0].set_ylabel('Index')
axes[2, 0].set_xlabel('Year')
axes[2, 0].grid(True, alpha=0.3)

# Consumer Sentiment
axes[2, 1].plot(df_combined['year'], df_combined['Consumer_Sentiment'], marker='o', linewidth=2, color='brown')
axes[2, 1].set_title('Consumer Sentiment', fontweight='bold')
axes[2, 1].set_ylabel('Index')
axes[2, 1].set_xlabel('Year')
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Time series plots - Key Economic Indicators
fig, axes = plt.subplots(3, 2, figsize=(16, 12))

# Satellite detections
axes[0, 0].plot(df_combined['year'], df_combined['satellite_detections'], marker='o', linewidth=2, color='steelblue')
axes[0, 0].set_title('Satellite Activity (Ports & Retail)', fontweight='bold')
axes[0, 0].set_ylabel('Total Detections')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axvspan(2020, 2021, alpha=0.2, color='red', label='COVID Period')

# GDP
axes[0, 1].plot(df_combined['year'], df_combined['GDP'], marker='o', linewidth=2, color='green')
axes[0, 1].set_title('Gross Domestic Product', fontweight='bold')
axes[0, 1].set_ylabel('GDP (Billions)')
axes[0, 1].grid(True, alpha=0.3)

# Unemployment Rate
axes[1, 0].plot(df_combined['year'], df_combined['Unemployment'], marker='o', linewidth=2, color='red')
axes[1, 0].set_title('Unemployment Rate', fontweight='bold')
axes[1, 0].set_ylabel('Rate (%)')
axes[1, 0].grid(True, alpha=0.3)

# Retail Sales
axes[1, 1].plot(df_combined['year'], df_combined['Retail_Sales'], marker='o', linewidth=2, color='purple')
axes[1, 1].set_title('Retail Sales', fontweight='bold')
axes[1, 1].set_ylabel('Sales (Millions)')
axes[1, 1].grid(True, alpha=0.3)

# Industrial Production
axes[2, 0].plot(df_combined['year'], df_combined['Industrial_Production'], marker='o', linewidth=2, color='orange')
axes[2, 0].set_title('Industrial Production Index', fontweight='bold')
axes[2, 0].set_ylabel('Index')
axes[2, 0].set_xlabel('Year')
axes[2, 0].grid(True, alpha=0.3)

# Consumer Sentiment
axes[2, 1].plot(df_combined['year'], df_combined['Consumer_Sentiment'], marker='o', linewidth=2, color='brown')
axes[2, 1].set_title('Consumer Sentiment Index', fontweight='bold')
axes[2, 1].set_ylabel('Index')
axes[2, 1].set_xlabel('Year')
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Correlation: Satellite Activity vs Key Indicators")
print("="*60)
correlations = df_combined[['satellite_detections', 'GDP', 'Unemployment', 
                             'Retail_Sales', 'Industrial_Production', 
                             'Consumer_Sentiment']].corr()['satellite_detections'].sort_values(ascending=False)
print(correlations)

In [ ]:
# Create lagged features
df_features = df_combined.copy()

# Lag satellite activity (previous year)
df_features['satellite_lag1'] = df_features['satellite_detections'].shift(1)

# Year-over-year changes
df_features['satellite_yoy_change'] = df_features['satellite_detections'].pct_change()
df_features['gdp_yoy_change'] = df_features['GDP'].pct_change()

# Drop first row (NaN from lag)
df_features = df_features.dropna()

print("✅ Feature Engineering Complete")
print(f"   Final shape: {df_features.shape}")
display(df_features)

## 6. Model Training (Next Steps)

**TODO:** Implement forecasting models:
1. Linear Regression (baseline)
2. XGBoost
3. LSTM (if converting to monthly)
4. Prophet (for time series)

**Target Variable:** Predict GDP, Retail Sales, or Industrial Production using satellite activity as leading indicator.

In [ ]:
# Placeholder for model training
print("🚧 Model training section - TO BE IMPLEMENTED")
print("\nSuggested approach:")
print("1. Define target variable (e.g., GDP growth)")
print("2. Split data: Train (2017-2022), Test (2023-2024)")
print("3. Train models: Linear Regression, XGBoost, LSTM")
print("4. Evaluate: RMSE, MAE, R²")
print("5. Make predictions for future periods")

## 7. Save Processed Data

In [ ]:
# Save combined dataset
output_path = PROJECT_ROOT / "data" / "features" / "combined_satellite_economic.csv"
df_features.to_csv(output_path, index=False)
print(f"✅ Saved combined dataset: {output_path}")

print("\n🎯 Ready for model training!")